In [135]:
import pandas as pd
import numpy as np

In [136]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [137]:
train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [138]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [139]:
train["train"]=0
test["train"]=1
combine = train.append(test, ignore_index=True)

In [140]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [141]:
# remove twitter handles (@user)
combine['tweet'] = np.vectorize(remove_pattern)(combine['tweet'], "@[\w]*|#[\w]*")

In [142]:
combine['tweet'] = combine['tweet'].str.replace("[^a-zA-Z]", " ")

In [143]:
combine

,id,label,tweet,train
0,1,0.0,when a father is dysfunctional and is so sel...,0
1,2,0.0,thanks for credit i can t use cause they do...,0
2,3,0.0,bihday your majesty,0
3,4,0.0,i love u take with u all the time in ur ...,0
4,5,0.0,factsguide society now,0
...,...,...,...,...
49154,49155,NaN,thought factory left right polarisation ...,1
49155,49156,NaN,feeling like a mermaid,1
49156,49157,NaN,today in omg amp used words like asse...,1
49157,49158,NaN,happy at work conference right mindset leads...,1


In [144]:
train=combine[combine["train"]==0]
test=combine[combine["train"]==1]

In [146]:
test.drop(["label"],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [157]:
x_train=train["tweet"]
y_train=train["label"]

x_test=test["tweet"]

In [148]:
!pip install sentencepiece

In [110]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [126]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [127]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tokenization

In [128]:
def bert_encode(texts, tokenizer, max_len=100):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [129]:
def build_model(bert_layer, max_len=100):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [130]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [158]:
train_input = bert_encode(x_train, tokenizer, max_len=50)
test_input = bert_encode(x_test, tokenizer, max_len=50)

In [132]:
model = build_model(bert_layer, max_len=50)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [133]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

In [134]:
train_history = model.fit(
    train_input, tf.convert_to_tensor(y_train, dtype=tf.float32),
    validation_split=0.2,
    epochs=2,
    batch_size=16
)

model.save('model.h5')

Epoch 1/2
1599/1599 [==============================] - 1904s 1s/step - loss: 0.1719 - accuracy: 0.9401 - val_loss: 0.1455 - val_accuracy: 0.9487
Epoch 2/2
1599/1599 [==============================] - 1902s 1s/step - loss: 0.0914 - accuracy: 0.9669 - val_loss: 0.1364 - val_accuracy: 0.9517
